In [1]:
import heapq

In [3]:
class HamiltonianCycleFinder:
    def __init__(self, adjacency_matrix):
        self.graph = adjacency_matrix
        self.num_vertices = len(adjacency_matrix)
        self.start_vertex = 0

    def find_hamiltonian_cycle(self):
        path = [self.start_vertex]
        visited = {self.start_vertex}
        current_cost = 0
        estimated_total_cost = self.estimate_remaining_cost(visited)

        priority_queue = []
        heapq.heappush(priority_queue, (estimated_total_cost, current_cost, path, visited))

        best_cycle = None
        best_cycle_cost = float("inf")

        while priority_queue:
            total_estimate, cost_so_far, path_so_far, visited_nodes = heapq.heappop(priority_queue)
            last_node = path_so_far[-1]

            # If all vertices are visited, check if returning to start is possible
            if len(path_so_far) == self.num_vertices:
                if self.graph[last_node][self.start_vertex] > 0:
                    cycle_cost = cost_so_far + self.graph[last_node][self.start_vertex]
                    if cycle_cost < best_cycle_cost:
                        best_cycle = path_so_far + [self.start_vertex]
                        best_cycle_cost = cycle_cost
                continue

            # Explore neighbors
            for neighbor in range(self.num_vertices):
                if self.graph[last_node][neighbor] > 0 and neighbor not in visited_nodes:
                    new_path = path_so_far + [neighbor]
                    new_visited = visited_nodes | {neighbor}
                    new_cost = cost_so_far + self.graph[last_node][neighbor]
                    new_estimate = new_cost + self.estimate_remaining_cost(new_visited)
                    heapq.heappush(priority_queue, (new_estimate, new_cost, new_path, new_visited))

        return best_cycle, best_cycle_cost if best_cycle else (None, None)

    def estimate_remaining_cost(self, visited):
        # Simple heuristic: assume at least min edge cost * remaining vertices
        remaining_vertices = set(range(self.num_vertices)) - visited
        if not remaining_vertices:
            return 0
        min_edge_weight = float("inf")
        for u in range(self.num_vertices):
            for v in range(self.num_vertices):
                if u != v and self.graph[u][v] > 0:
                    min_edge_weight = min(min_edge_weight, self.graph[u][v])
        return len(remaining_vertices) * (min_edge_weight if min_edge_weight != float("inf") else 1)

In [10]:
def build_adjacency_matrix(num_vertices, edge_list):
    matrix = [[0] * num_vertices for _ in range(num_vertices)]
    for u, v, weight in edge_list:
        matrix[u-1][v-1] = weight
        matrix[v-1][u-1] = weight  # undirected graph
    return matrix


def main():
    num_vertices = 4
    edge_list = [
        (1, 2, 10),
        (2, 3, 15),
        (3, 4, 25),
        (4, 1, 25),
        (1, 3, 35),
        (2, 4, 30)
    ]

    adjacency_matrix = build_adjacency_matrix(num_vertices, edge_list)
    solver = HamiltonianCycleFinder(adjacency_matrix)
    cycle, min_cycle_cost = solver.find_hamiltonian_cycle()

    if cycle:
        print("Hamiltonian cycle found with minimum cost:")
        print(" -> ".join(str(v + 1) for v in cycle))
        print("Total cost:", min_cycle_cost)
    else:
        print("No Hamiltonian cycle exists.")

In [11]:
if __name__ == "__main__":
    main()

Hamiltonian cycle found with minimum cost:
1 -> 2 -> 3 -> 4 -> 1
Total cost: 75
